In [186]:
import numpy as np 
import pandas as pd
import requests as rq
from datetime import date,datetime
from fastai.tabular.all import *
api_key="*****************"
weather_key="************************"

In [ ]:
eventType=""
eventDate=""
systems=["austintx","caltrans","ddot_captop","deldot","dfw511","fdot","gdot",
         "hococad","ildot","inrix","iowadot","massdot","mdot_chart","midot",
         "mndot","modot","mstraffic","ncdot","nhdot","njdot","ny511","ohdot",
         "ordot",'padot','paturnpike','pgcocad','pgcotrip','ridot','ritis_input',
         'scdot','txdot','vdot','wisdot','wisdot_cad','wmata','wsdot']

columns=['Zipcode', 'Start_Lat', 'Start_Lng', 'State',
       'Temperature', 'Humidity', 'Pressure', 'Wind_Speed',
       'Month', 'Day', 'Hour', 'Dayofweek', 'Weekend', 'Weather_id']
s="all"
file=""
u=0
df=pd.DataFrame()
counter=0
while True:
    u+=1
    if u==len(systems):u=0
    response=rq.get("http://api.trafficview.org/event/?api-key="+api_key,params={"system":systems[u],"format":"rf-json"})
    try:
        json=response.json()['trafficEvents']['events']
        
    except:
        #no accidents
        continue
    
    try:
        file=pd.read_csv(s+"_raw.csv").drop(columns='Unnamed: 0')
        df=file.copy(True)
        
    except:
        file=-1
    l=[]
    
    for i in range(len(json)):
        eventType=json[i]['head']['eventType']['type']
        if eventType=="roadwork": continue
        try:
            eventDate=json[i]['head']['dates']['created']
        except: eventDate=json[i]['head']['dates']['started']
        try:eventDate=datetime.fromisoformat(eventDate)
        except:continue
        year=eventDate.year
        month=eventDate.month
        day=eventDate.day
        hour=eventDate.hour
        dayofweek=eventDate.isoweekday()
        weekend=eventDate.isoweekday()>=6
        locations=json[i]['locations']
        
        for z in range(len(locations)):
            lat=""
            lon=""
            weth=""
            '''try: prefix=locations[z]['pointLocation']['onAddress']['road']['prefix'] 
            except:prefix=""
            try: route=locations[z]['pointLocation']['onAddress']['road']['route'] 
            except:route=""
            try:direction=locations[z]['pointLocation']['onAddress']['road']['direction'] 
            except:direction=""'''
            try:
                pos=locations[z]['pointLocation']['Point']['pos']
                lat=pos['lat']    
                lon=pos['lon']
                city=locations[z]['pointLocation']['onAddress']['city']
                zipcode=locations[z]['pointLocation']['onAddress']['zipcode']
                county=locations[z]['pointLocation']['onAddress']['county']
                state=locations[z]['pointLocation']['onAddress']['state']
                if state.lower() in ['new york','ny','newyork']:state="NY"
                elif state.lower() in ['rhode island','ri','rhodeisland']:state="RI"
                elif state.lower() in ['new hamsphire','nh','newhamsphire']:state="NH"
                elif state.lower() in ['pennsylvania','pa']:state="PA"
                elif state.lower() in ['massachusetts','ma']:state="MA"
                
            except: continue
            counter+=1
            weth=rq.get("https://api.openweathermap.org/data/2.5/onecall/timemachine",
               params={"appid":weather_key,"lat":lat,"lon":lon,"units":"imperial","dt":int(eventDate.timestamp())})
            if (counter>50):
                counter=0
                time.sleep(10)
                
            if (weth.status_code==400):
                #print("SKIP")
                #Weather free API only get data for the past 5 days. If it's older then it will return an error
                continue

            temp=weth.json()['current']['temp']
            pressure=float(weth.json()['current']['pressure'])/33.86
            humidity=weth.json()['current']['humidity']
            wind_speed=weth.json()['current']['wind_speed']
            weather=weth.json()['current']['weather'][0]['id']
            l.append([zipcode,lat,lon,state,temp,humidity,pressure,wind_speed,month,day,hour,dayofweek,weekend,weather])
            r=pd.DataFrame(l,columns=columns)
            df=pd.concat([df,r],axis=0,ignore_index=True)
    if df.shape[0]==0: continue #no accidents
    df.columns=columns
    df.drop_duplicates(inplace=True)
    df.to_csv(s+"_raw.csv")
    learn=load_learner(s+".pkl")
    to = learn.dls.train_ds.new(df)
    to.process()
    dl = TabDataLoader(to)
    a,b=learn.get_preds(dl=dl)
    l=[]
    for i in a:
        l.append(i[1].tolist()>=0.5)
    df['Severity']=l
    df.to_csv(s+"_predict.csv")
    print("Created data for ",state,"successfully !")
    time.sleep(10)  

Created data for  California successfully !


Created data for  District of Columbia successfully !


Created data for  Delaware successfully !


Created data for  Delaware successfully !


Created data for  Illinois successfully !
